<a href="https://colab.research.google.com/github/Arshapjoy/Movie_Recommendation_System_Project/blob/main/notebooks/3_rec_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:

combined_df=pd.read_csv("/content/drive/MyDrive/MCA_project/Final/combined_cleaned.csv")

In [3]:
combined_df.head()

,id,title,audienceScore,tomatoMeter,rating,ratingContents,runtimeMinutes,genre,originalLanguage,director,writer,distributor,reviewId,reviewText,scoreSentiment,originalScore_normalized
0,love_lies,"Love, Lies",43.0,66.920339,Unknown,Unknown,120.0,['drama'],Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,2739073.0,Though let down by its routine love triangle n...,POSITIVE,8.0
1,love_lies,"Love, Lies",43.0,66.920339,Unknown,Unknown,120.0,['drama'],Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,2333658.0,"While not perfect, Love, Lies is a worthy disc...",POSITIVE,7.0
2,adrift_2018,Adrift,65.0,69.000000,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",120.0,"['adventure', 'drama', 'romance']",English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",STX Films,102694850.0,This is nowhere near the level of other great ...,POSITIVE,6.0
3,adrift_2018,Adrift,65.0,69.000000,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",120.0,"['adventure', 'drama', 'romance']",English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",STX Films,2816011.0,"For almost the entire runtime, the only people...",NEGATIVE,7.0
4,adrift_2018,Adrift,65.0,69.000000,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",120.0,"['adventure', 'drama', 'romance']",English,Baltasar Kormákur,"Aaron Kandell,Jordan Kandell,David Branson Smith",STX Films,2772153.0,What is meant to be a voyage of self-discovery...,NEGATIVE,4.0


In [4]:
combined_df["genre"].unique()

array(["['drama']", "['adventure', 'drama', 'romance']",
       "['crime', 'drama']", ...,
       "['action', 'animation', 'drama', 'scifi']",
       "['crime', 'documentary', 'history', 'sportsandfitness']",
       "['drama', 'romance', 'sportsandfitness']"], dtype=object)

In [5]:
combined_df.isnull().sum()

id                          0
title                       0
audienceScore               0
tomatoMeter                 0
rating                      0
ratingContents              0
runtimeMinutes              0
genre                       0
originalLanguage            0
director                    0
writer                      0
distributor                 0
reviewId                    0
reviewText                  0
scoreSentiment              0
originalScore_normalized    0
dtype: int64

In [6]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993876 entries, 0 to 993875
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id                        993876 non-null  object 
 1   title                     993876 non-null  object 
 2   audienceScore             993876 non-null  float64
 3   tomatoMeter               993876 non-null  float64
 4   rating                    993876 non-null  object 
 5   ratingContents            993876 non-null  object 
 6   runtimeMinutes            993876 non-null  float64
 7   genre                     993876 non-null  object 
 8   originalLanguage          993876 non-null  object 
 9   director                  993876 non-null  object 
 10  writer                    993876 non-null  object 
 11  distributor               993876 non-null  object 
 12  reviewId                  993876 non-null  float64
 13  reviewText                993876 non-null  o

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
def preprocess_data(df):
    df['combined_features'] = df.apply(lambda x: f"{x['title']} {x['genre']} {x['director']} {x['writer']} {x['distributor']}", axis=1)
    return df

In [9]:
combined_df = preprocess_data(combined_df)

In [10]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)  # Limit the number of features for efficiency
tfidf_matrix = tfidf.fit_transform(combined_df['combined_features'])

In [ ]:
# Use sparse matrix for cosine similarity computation
cosine_sim = cosine_similarity(tfidf_matrix, dense_output=False)

In [ ]:
# Recommendations function
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = combined_df[combined_df['title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = cosine_sim[idx].toarray().flatten()

    # Get the indices of the top 10 most similar movies
    sim_indices = np.argsort(-sim_scores)[1:11]

    # Return the top 10 most similar movies
    return combined_df['title'].iloc[sim_indices]

In [ ]:
# Example usage
recommendations = get_recommendations('The Matrix')
print(recommendations)